## 1、将图片逐帧转换回视频

In [ ]:
import cv2
import os
from datetime import datetime
 
# print(cv2.__version__)
 
def frames_to_video(save_path, frames_path):
    # fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    fps = 25
    video_width, video_height = 1920, 1080
    videoWriter = cv2.VideoWriter(save_path, fourcc, fps, (video_width, video_height))
    # videoWriter = cv2.VideoWriter(save_path, -1, fps, (video_width, video_height))
    imgs = glob.glob(frames_path + "/*.jpg")
    # print(imgs)
    # frames_num = len(imgs)
    for i in range(2851,2894):
        if os.path.isfile("%s/%d.jpg"%(frames_path, i)):
            # print(i)
            frame = cv2.imread("%s/%d.jpg"%(frames_path, i))
            # print(frame.shape)
            videoWriter.write(frame)
    videoWriter.release()
    return
 
 
if __name__ == '__main__':

    frames_to_video("jian_1_25.mp4", 'jian_1_25_frames')

In [31]:
def visdrone2yolo(dir):
    from PIL import Image
    from tqdm import tqdm
    def convert_box(size, box):
        # Convert VisDrone box to YOLO xywh box
        dw = 1. / size[0]
        dh = 1. / size[1]
        return (box[0] + box[2] / 2) * dw, (box[1] + box[3] / 2) * dh, box[2] * dw, box[3] * dh
    (dir / 'labels').mkdir(parents=True, exist_ok=True)  # make labels directory
    pbar = tqdm((dir / 'annotations').glob('*.txt'), desc=f'Converting {dir}')
    for f in pbar:
        img_size = Image.open((dir / 'images' / f.name).with_suffix('.jpg')).size
        lines = []
        with open(f, 'r') as file:  # read annotation.txt
            for row in [x.split(',') for x in file.read().strip().splitlines()]:
                cls = int(row[5]) - 1
                if row[4] == '0' : # VisDrone 'ignored regions' class 0
                    continue
                elif cls in (0,1,6,7) :
                    continue              
                box = convert_box(img_size, tuple(map(int, row[:4])))
                lines.append(f"{cls} {' '.join(f'{x:.6f}' for x in box)}\n")
                with open(str(f).replace(os.sep + 'annotations' + os.sep, os.sep + 'labels' + os.sep), 'w') as fl:
                    fl.writelines(lines)  # write label.txt           

In [32]:
from utils.general import download, os, Path

dir = Path('../VisDrone') 

for d in 'VisDrone2019-DET-train', 'VisDrone2019-DET-val', 'VisDrone2019-DET-test-dev':
  visdrone2yolo(dir / d)  # convert VisDrone annotations to YOLO labels    

Converting ../VisDrone/VisDrone2019-DET-train: 6471it [00:11, 546.98it/s]
Converting ../VisDrone/VisDrone2019-DET-val: 548it [00:01, 466.03it/s]
Converting ../VisDrone/VisDrone2019-DET-test-dev: 1610it [00:02, 651.50it/s]
